In [582]:
import numpy as np

In [583]:
n_vertices = 6
h = 50                                                      # mm
l_edge = 20                                                 # mm
alpha = 540                                                  # degrees

In [584]:
xmax = 220                                                  # mm
ymax = 220                                                  # mm
zmax = 300                                                  # mm

In [585]:
t_bed = 60                                                  # degrees C
t_ext = 210                                                 # degrees C

h_layer = 0.2                                               # mm
feed_sec = 40                                               # mm/s
t_feed_sec = 200                                            # mm/s

feed = feed_sec * 60                                        # mm/min
t_feed = t_feed_sec * 60                                    # mm/min

max_a = [500.00, 500.00, 100.00, 5000.00]
max_feed = [500.00, 500.00, 10.00, 50.00]
accs = [500.00, 1000.00, 500.00]
jerk = [8.00, 8.00, 0.40, 5.00]

v_fan = 100                                                 # %
d_nozzle = 0.4                                              # mm
d_filament = 1.75                                           # mm

In [586]:
text = open('Ex_Files/Ex1.gcode', 'w')
text.write(";This Gcode was generated with GPyCode\n")
text.write(";The shape has {} vertices\n".format(int(n_vertices)))
text.write(";The print has a layer height of {} mm and the feed speed is {} mm/s\n".format(h_layer, feed_sec))


70

In [587]:
text.write('M140 S{}\n'.format(t_bed))                      # Set bed temperature without waiting
text.write('M104 S{}\n'.format(t_ext))                      # Set extruder temperature without waiting
text.write('M105\n')
text.write('M190 S{}\n'.format(t_bed))                      # Set bed temperature and wait
text.write('M109 S{}\n'.format(t_ext))                      # Set extruder temperature and wait

10

In [588]:
text.write('M82\n')
text.write('M201 X{} Y{} Z{} E{}\n'.format(*max_a))         # Set max acceleration for all axis
text.write('M203 X{} Y{} Z{} E{}\n'.format(*max_feed))      # Set max feed for all axis
text.write('M204 P{} R{} T{}\n'.format(*accs))              # Set print, retract and travel accelerations
text.write('M205 X{} Y{} Z{} E{}\n'.format(*jerk))          # Set Jerk

text.write('M220 S100\n')                                   # Feed multiplier
text.write('M221 S100\n')                                   # Flow rate multiplier

text.write('G28\n')                                         # Auto home
text.write('G92 E0\n')                                      # Reset extruder to 0
text.write('M107\n')                                        # Disable fan for first layer


5

In [589]:
xc = xmax / 2
yc = ymax / 2
text.write('G0 F{} X{} Y{} Z2.00\n'.format(t_feed, xc, yc))

30

In [590]:
n_layers = int(h / h_layer)
z_layers = [round(h_layer + (i * h_layer), 2) for i in range(n_layers)]

if (h / h_layer) != int(h / h_layer):
    n_layers += 1
    z_layers.append(round(h, 2))

alpha_layers = [alpha * (i / (n_layers - 1)) for i in range(n_layers)]

In [591]:
theta_0 = [i * (360 / n_vertices) for i in range(n_vertices)]
r = (l_edge / 2) / np.cos((180 * (n_vertices - 2)) / n_vertices)

In [592]:
def calcExt(l, d_noz, d_fil, h_lay):
    return (d_noz * h_lay * l) / (np.pi * ((d_fil / 2)**2))

In [593]:
E = 0

for layer in range(n_layers):
    theta = [(alpha_layers[layer] + theta_0[i]) * (np.pi / 180) for i in range(len(theta_0))]
    x = [xc + (r * np.cos(th)) for th in theta]
    y = [yc + (r * np.sin(th)) for th in theta]

    if layer == 1:
        text.write('M106 S128\n')
    elif layer == 2:
        text.write('M106 S255\n')

    text.write('G0 F{} X{} Y{} Z{}\n'.format(t_feed, round(x[0], 3), round(y[0], 3), z_layers[layer]))

    for point in range(1, n_vertices):
        dl = np.sqrt((x[point] - x[point - 1])**2 + (y[point] - y[point - 1])**2)
        E += calcExt(dl, d_nozzle, d_filament, h_layer)
        text.write('G1 F{} X{} Y{} E{}\n'.format(feed, round(x[point], 3), round(y[point], 3), round(E, 5)))
    
    dl = np.sqrt((x[0] - x[-1])**2 + (y[0] - y[-1])**2)
    E += calcExt(dl, d_nozzle, d_filament, h_layer)
    text.write('G1 F{} X{} Y{} E{}\n'.format(feed, round(x[0], 3), round(y[0], 3), round(E, 5)))
    

In [594]:
text.write('G0 Z{}\n'.format(z_layers[-1] + 20))
text.write('G28 X Y\n')
text.write('M107\n')
text.write('M140 S0\n')
text.write('M104 S0\n')
text.write('M84 X Y Z E\n')

12

In [595]:
text.close()